## ISE 535 Final Project

In [279]:
import numpy as np
import pandas as pd
import requests
import ipywidgets
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

layout = widgets.Layout(width='400px', height='30px')
pd.options.display.float_format = "{:.2f}".format
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

from ipywidgets import interact, IntSlider, interactive, widgets, interact_manual, fixed
from pandas import DataFrame
from IPython.display import HTML

import folium
from folium import plugins


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Function to Generate Latest Active Cases per Country


In [280]:
def countryData(country):
    url1 = "https://api.covid19api.com/dayone/country/"
    country = country
    url = url1+country
    payload={}
    headers = {'X-Access-Token': 'a3c98472-2928-4752-9d71-083ce072213c'}
    response = requests.request("GET", url, headers=headers, data=payload)
    data = response.json()
    df = DataFrame(data)
    return df

### Summary of All Countries

In [308]:
url = "https://api.covid19api.com/countries"
payload={}
headers = {
    'X-Access-Token': 'a3c98472-2928-4752-9d71-083ce072213c'
}
response = requests.request("GET", url, headers=headers, data=payload)
data = response.json()
df = DataFrame(data)
df.head()

,Country,Slug,ISO2
0,Netherlands Antilles,netherlands-antilles,AN
1,Nigeria,nigeria,NG
2,Palau,palau,PW
3,Palestinian Territory,palestine,PS
4,Bermuda,bermuda,BM


### Latest Cases for South East Asian Countries

In [282]:
seCountries = ["brunei","myanmar","cambodia","timor-leste","indonesia","malaysia", 
               "philippines","singapore","thailand","vietnam"]

In [283]:
def latestCountry(country):
    url1 = "https://api.covid19api.com/dayone/country/"
    country = country
    url = url1+country
    payload={}
    headers = {'X-Access-Token': 'a3c98472-2928-4752-9d71-083ce072213c'}
    response = requests.request("GET", url, headers=headers, data=payload)
    data = response.json()
    df = DataFrame(data)
    return df.tail(1)

In [284]:
dataSE = []

for country in seCountries:
    df = latestCountry(country)
    columns = list(df)
    values = df.iloc[0]
    zipped = zip(columns, values)
    a_dictionary = dict(zipped)
    dataSE.append(a_dictionary)

In [285]:

dataSE = DataFrame(dataSE).sort_values('Country')
dataSE

,ID,Country,CountryCode,Province,City,CityCode,Lat,Lon,Confirmed,Deaths,Recovered,Active,Date
0,1f8dc59c-f5d8-41c9-9f6a-d11711019ad4,Brunei Darussalam,BN,,,,4.54,114.73,13321,89,0,13232,2021-11-03T00:00:00Z
2,8f87c0fb-4d80-44db-a82a-163514829746,Cambodia,KH,,,,12.57,104.99,118787,2807,0,115980,2021-11-03T00:00:00Z
4,19d9629f-b0cc-42f1-bb21-560e8d0c1794,Indonesia,ID,,,,-0.79,113.92,4245373,143457,0,4101916,2021-11-03T00:00:00Z
5,59865321-4035-4184-bf45-71ac05e22acb,Malaysia,MY,,,,4.21,101.98,2481339,29045,0,2452294,2021-11-02T00:00:00Z
1,7e017844-ff5d-46e5-8080-26e0500c9c79,Myanmar,MM,,,,21.91,95.96,502024,18728,0,483296,2021-11-03T00:00:00Z
6,c522d171-4753-490a-a9c9-98543ca7692b,Philippines,PH,,,,12.88,121.77,2792656,43404,0,2749252,2021-11-03T00:00:00Z
7,48b63945-f707-4d49-9540-aed080b51661,Singapore,SG,,,,1.35,103.82,204340,430,0,203910,2021-11-03T00:00:00Z
8,cc39aba2-695c-4133-bfe4-c720581dcd88,Thailand,TH,,,,15.87,100.99,1935442,19394,0,1916048,2021-11-03T00:00:00Z
3,f557da33-c101-4980-bdbf-78c87a7cce24,Timor-Leste,TL,,,,-8.87,125.73,19790,122,0,19668,2021-11-03T00:00:00Z
9,fd01268e-9b44-4afe-bf17-a6b5bfb3be1c,Viet Nam,VN,,,,14.06,108.28,932357,22205,0,910152,2021-11-03T00:00:00Z


### Latest Cases Table and Plot

In [286]:
#how to select columns and highlight dataframe using gradient background
dataFilter = dataSE[['Country','Confirmed','Deaths', 'Active']].sort_values('Confirmed', ascending=False).reset_index(drop=True)
dataFilter.style.background_gradient(cmap='Reds')


,Country,Confirmed,Deaths,Active
0,Indonesia,4245373,143457,4101916
1,Philippines,2792656,43404,2749252
2,Malaysia,2481339,29045,2452294
3,Thailand,1935442,19394,1916048
4,Viet Nam,932357,22205,910152
5,Myanmar,502024,18728,483296
6,Singapore,204340,430,203910
7,Cambodia,118787,2807,115980
8,Timor-Leste,19790,122,19668
9,Brunei Darussalam,13321,89,13232


In [316]:
variableSel = widgets.RadioButtons(
    options=['Confirmed', 'Deaths', 'Active'],
    value='Confirmed',
    description='Variable:',
    disabled=False
)

def scatterLatest(variable):
    fig = px.scatter(dataFilter, x='Country', y=variable, size=variable,
                color='Country', hover_name='Country', size_max=50,
                title=f"Latest Covid-19 {variable} Cases as of {dataSE['Date'][0]}")
    return fig.show()
    
interact(scatterLatest,variable=variableSel)


interactive(children=(RadioButtons(description='Variable:', options=('Confirmed', 'Deaths', 'Active'), value='…

<function __main__.scatterLatest(variable)>

In [287]:
map = folium.Map(location=[15,100], zoom_start=4, tiles='Stamenterrain')

for lat, lon, value, name in zip(dataSE['Lat'],dataSE['Lon'],dataSE['Active'],dataSE['Country']):
    folium.CircleMarker([lat, lon],radius=value*0.8, popup=(str(name).capitalize())).add_to(map)
#save map as hmtl, attach html to the readme file in gitpages

In [511]:
map.save(outfile= "map.html")

[maplink]("map.html")

### Big Data Set to Use with Widgets

In [288]:
#reading-in data
bigData = []
for country in seCountries:
    df = countryData(country)
    bigData.append(df)
bigData = pd.concat(bigData)

In [289]:
#cleaning and filtering
bigData = bigData[['Country', 'CountryCode','Confirmed','Deaths','Recovered','Active','Date']]
bigData['Date'] = pd.to_datetime(bigData.Date, format='%Y-%m-%d %H:%M:%S')
bigData['Date'] = bigData['Date'].dt.strftime('%Y-%m-%d')
bigData['Date']= pd.to_datetime(bigData["Date"])
bigData = bigData.reset_index(drop=True)

### Widgets

In [292]:
countrySel = widgets.Dropdown(
    options=['Brunei Darussalam', 'Cambodia', 'Indonesia','Malaysia','Myanmar',
            'Philippines', 'Singapore', 'Thailand', 'Timor-Leste','Viet Nam'],
    value='Brunei Darussalam',
    description='Country:',
    disabled=False,
)

variableSel = widgets.Select(
    options=['Confirmed', 'Deaths', 'Recovered', 'Active'],
    value='Confirmed',
    description='Variable:',
    disabled=False
)

#Dates entered are exclusive
start = datetime.datetime(2020, 6, 4)
end = datetime.datetime(2020, 6, 14)
startDateSel = widgets.DatePicker(description='Start Date:', disabled=False, value=start)
endDateSel = widgets.DatePicker(description='End Date:', disabled=False, value=end)

### Widgets Interact with Function to Filter Dataframe

In [293]:
def filter(country, variable, startDate, endDate):
    filter1 = bigData[bigData['Country'] == countrySel.value]
    filter2 = filter1[['Country', 'CountryCode', variableSel.value, 'Date']]
    filter3 = filter2[(filter2['Date']>pd.to_datetime(startDateSel.value)) & 
                   (filter2['Date']<pd.to_datetime(endDateSel.value))]
    return filter3

interact(filter, country=countrySel, variable=variableSel, startDate=startDateSel, endDate=endDateSel)

interactive(children=(Dropdown(description='Country:', options=('Brunei Darussalam', 'Cambodia', 'Indonesia', …

<function __main__.filter(country, variable, startDate, endDate)>

### Line Graphs

In [318]:
bigData.head()

,Country,CountryCode,Confirmed,Deaths,Recovered,Active,Date
0,Brunei Darussalam,BN,1,0,0,1,2020-03-09
1,Brunei Darussalam,BN,1,0,0,1,2020-03-10
2,Brunei Darussalam,BN,11,0,0,11,2020-03-11
3,Brunei Darussalam,BN,11,0,0,11,2020-03-12
4,Brunei Darussalam,BN,37,0,0,37,2020-03-13


In [317]:
import plotly.express as px

In [339]:
data = countryData('Philippines')

,ID,Country,CountryCode,Province,City,CityCode,Lat,Lon,Confirmed,Deaths,Recovered,Active,Date
0,72fde949-3746-46df-9292-6040e97c5286,Philippines,PH,,,,12.88,121.77,1,0,0,1,2020-01-30T00:00:00Z
1,949a266b-8775-4b1e-80a1-b88ae6aa3ab0,Philippines,PH,,,,12.88,121.77,1,0,0,1,2020-01-31T00:00:00Z
2,a137de21-314d-49a0-90fb-4485b1fa7a5f,Philippines,PH,,,,12.88,121.77,1,0,0,1,2020-02-01T00:00:00Z
3,4692103d-17cd-40ea-a70b-a6aac04fdb40,Philippines,PH,,,,12.88,121.77,2,1,0,1,2020-02-02T00:00:00Z
4,0052d936-7483-4eeb-8e99-6109a40cf764,Philippines,PH,,,,12.88,121.77,2,1,0,1,2020-02-03T00:00:00Z
5,3635bbcb-5486-4c61-a2cc-96212257a101,Philippines,PH,,,,12.88,121.77,2,1,0,1,2020-02-04T00:00:00Z
6,5ffdd93f-75b3-4954-bac0-bb0981404eb5,Philippines,PH,,,,12.88,121.77,2,1,0,1,2020-02-05T00:00:00Z
7,524c58ec-ad47-4d56-b0f2-6e49fd2407a2,Philippines,PH,,,,12.88,121.77,2,1,0,1,2020-02-06T00:00:00Z
8,f3cec544-237d-4e81-924e-5d8cc4143b37,Philippines,PH,,,,12.88,121.77,3,1,0,2,2020-02-07T00:00:00Z
9,72ecef4b-d6d6-4156-873e-3cc26bc61472,Philippines,PH,,,,12.88,121.77,3,1,0,2,2020-02-08T00:00:00Z


In [354]:
data['DailyCases'] = data['Confirmed'].diff()
data = data.dropna()
data['DailyCases'] = data['DailyCases'].astype(int)

,ID,Country,CountryCode,Province,City,CityCode,Lat,Lon,Confirmed,Deaths,Recovered,Active,Date,Daily Cases,DailyCases
639,bc489855-59ea-4e1f-b292-6df106bc4ceb,Philippines,PH,,,,12.88,121.77,2783896,43044,0,2740852,2021-10-30T00:00:00Z,3953.00,3953
640,41cbb455-c858-4500-ab59-9cd350e27021,Philippines,PH,,,,12.88,121.77,2787276,43172,0,2744104,2021-10-31T00:00:00Z,3380.00,3380
641,91d4b7e4-9c07-4c06-90e4-cb9e8f00979a,Philippines,PH,,,,12.88,121.77,2790375,43276,0,2747099,2021-11-01T00:00:00Z,3099.00,3099
642,67e476c2-557b-4c80-8566-0ea144c49257,Philippines,PH,,,,12.88,121.77,2792656,43404,0,2749252,2021-11-02T00:00:00Z,2281.00,2281
643,c522d171-4753-490a-a9c9-98543ca7692b,Philippines,PH,,,,12.88,121.77,2792656,43404,0,2749252,2021-11-03T00:00:00Z,0.00,0


In [355]:
fig = px.line(df, x=data['Date'], y=data['DailyCases'])
fig.show()
    
